# pyscipopt & sudoku

[![](https://pic.imgdb.cn/item/6532337cc458853aef5a44b3.png)](https://pic.imgdb.cn/item/6532337cc458853aef5a44b3.png)

(*上图: 号称难度系数最高数独*)

数独是一个非常有趣的数学游戏, 最为常见的形式为九宫格: **9 x 9**

- 每个格子填入 1 - 9范围的数字, 每个格子填入的数字都不相同
- 每个小宫格 3 x 3范围填入, 遵守上述规则
- 行列之间的数字填入, 遵守上述规则

数独这个问题的解法非常巧妙, 这里参考了[通过整数规划求解数独谜题：基于问题 - MATLAB & Simulink - MathWorks 中国](https://ww2.mathworks.cn/help/optim/ug/sudoku-puzzles-problem-based.html)提供的思路.

> 关键思路是将谜题从 9×9 正方形网格转换为一个由二元值（0 或 1）组成的 9×9×9 立方数组。将这个立方数组想象成由 9 个正方形网络上下堆叠在一起。顶部网格是数组的第一个正方形层，只要解或提示包含整数 1，则该层对应位置就为 1。只要解或提示包含整数 2，第二层对应位置就为 1。只要解或提示包含整数 9，第九层对应位置就为 1。

![img](https://pic3.zhimg.com/80/v2-2f23f1294d83604a768c45f9e26c097a_720w.webp)

(*图源*: [数学建模代码实战训练-5：数独里的整数线规模型构造分析 - 知乎](https://zhuanlan.zhihu.com/p/362008314), 作者版权所有)

以整数规划为视角创建数学模型:
$$
\\
\begin{aligned}
\text{min} & \quad 0                        &&                                       &&(1)\\ 
\text{s.t.} & \quad \sum_{k=1}^{9}x_{r,c,k} = 1,\quad &&\forall r,c = 1,\dots,9,       &&(2)\\ 
& \quad \sum_{r=1}^{9}x_{r,c,k}=1,  \quad &&\forall c,k = 1,\dots,9,                   &&(3)\\  
& \quad \sum_{c=1}^{9}x_{r,c,k}=1,  \quad &&\forall r,k = 1,\dots,9,                   &&(4)\\
& \quad \sum_{r'=1}^{3}\sum_{c'=1}^{3} x_{r'+r,c'+c,k} = 1,\quad&&\forall r,c \in \{0,3,6\},\ k = 1,\dots,9,                                                                           &&(5)\\   
& \quad x_{r,c,k}=1, &&\forall(r,c,k)\in C,                                            &&(6)\\ 
& \quad x_{r,c,k}\in \{0,1\}\quad &&\forall r,c,k = 1,\dots,9.                         &&(7)
\end{aligned} \\
\\
\begin{aligned}
&r, row, 行;\\
&c, column, 列\\
&k, 1 - 9的数字\\
&C, 已填入数字的坐标\\
\end{aligned}
\\
\begin{aligned}
&(1): 目标函数, 这里不需要设置, 常数0\\
&约束:\\
&(2): 每个格子填入一个数字 \\
&(3): 每行格子填入一个数字, 不重复 \\
&(4): 每列格子填入一个数字, 不重复\\
&(5): 每个小宫格内填入一个数字, 不重复\\
&(6): 已经填入的格子, 其值表示为:1\\
&(7): 决策变量的取值范围, 即0-1型\\
\end{aligned}
$$

In [1]:
import pyscipopt as opt

In [16]:
# 创建上述数独

matrix = [
    [8, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 3, 6, 0, 0, 0, 0, 0],
    [0, 7, 0, 0, 9, 0, 2, 0, 0],
    [0, 5, 0, 0, 0, 7, 0, 0, 0],
    [0, 0, 0, 0, 4, 5, 7, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 3, 0],
    [0, 0, 1, 0, 0, 0, 0, 6, 8],
    [0, 0, 8, 5, 0, 0, 0, 1, 0],
    [0, 9, 0, 0, 0, 0, 4, 0, 0]
]

In [17]:
# 普通参数

# matrix shape
N = 9
# litte matrix shape
n = 3
# 填入的数字
constants = [e for e in range(1, N + 1)]

```bash
Init docstring:
:param problemName: name of the problem (default 'model'), 模型名称
:param defaultPlugins: use default plugins? (default True)
:param sourceModel: create a copy of the given Model instance (default None)
:param origcopy: whether to call copy or copyOrig (default False), 数据复制操作相关?
:param globalcopy: whether to create a global or a local copy (default True)
:param enablepricing: whether to enable pricing in copy (default False)
:param createscip: initialize the Model object and creates a SCIP instance
:param threadsafe: False if data can be safely shared between the source and target problem, 线程安全
```

In [18]:
model = opt.Model("sudoku")

```bash
model.addVar

Docstring:
Create a new variable. Default variable is non-negative and continuous. 变量, 默认是非负连续型

:param name: name of the variable, generic if empty (Default value = ''), 变量名称
:param vtype: type of the variable: 'C' continuous, 'I' integer, 'B' binary, and 'M' implicit integer, 变量类型
(see https://www.scipopt.org/doc/html/FAQ.php#implicitinteger) (Default value = 'C'), 默认为连续型
:param lb: lower bound of the variable, use None for -infinity (Default value = 0.0), 下界, 使用None来表示负无穷
:param ub: upper bound of the variable, use None for +infinity (Default value = None), 上界, 使用None来表示正无穷
:param obj: objective value of variable (Default value = 0.0), 变量默认值
:param pricedVar: is the variable a pricing candidate? (Default value = False), 这两个参数是比较有意思的, 不确定其他的算法框架是否提供
:param pricedVarScore: score of variable in case it is priced, the higher the better (Default value = 1.0)
```

In [19]:
# 创建变量, 需要 9 * 9 * 9 = 729个决策变量
variables = {}
for r in range(N):
    for c in range(N):
        for k in constants:
            key = (r, c, k)
            variables[key] = model.addVar(
                name = '_'.join(str(e) for e in key),
                # 数据类型为布尔型, 即0, 1
                vtype = 'B'
            )
# 同时这里实现上述模型中的决策变量 0-1约束

```bash
model.addCons?
Add a linear or nonlinear constraint., 添加线性/非线性约束

:param cons: constraint object
:param name: the name of the constraint, generic name if empty (Default value = ''), 名称
:param initial: should the LP relaxation of constraint be in the initial LP? (Default value = True), 线性规划约束松弛变量是否初始化(通过引入松弛变量, 把原优化问题中的不等式约束转化为等式约束)
:param separate: should the constraint be separated during LP processing? (Default value = True) 在LP处理过程中约束是否应该分离?
:param enforce: should the constraint be enforced during node processing? (Default value = True) 在节点处理过程中是否应该强制执行约束?(
:param check: should the constraint be checked for feasibility? (Default value = True)是否检查约束的可行性
:param propagate: should the constraint be propagated during node processing? (Default value = True) 在节点处理过程中是否允许约束扩散
:param local: is the constraint only valid locally?  (Default value = False) 约束仅在本地?
:param modifiable: is the constraint modifiable (subject to column generation)? (Default value = False)约束是否可修改, 应用于列生成算法
:param dynamic: is the constraint subject to aging? (Default value = False) 约束衰减?
:param removable: should the relaxation be removed from the LP due to aging or cleanup? (Default value = False) 由于老化或清理，松弛物是否应从LP上去除
:param stickingatnode: should the constraint always be kept at the node where it was added, even if it may be  moved to a more global node? (Default value = False)
:return The added @ref scip#Constraint "Constraint" object.
```

In [20]:
# 约束1, 对已经填写了数字变量坐标添加1策略约束
for r in range(N):
    for c in range(N):
        if matrix[r][c] != 0:
            constrain = variables[r, c, matrix[r][c]] == 1
            model.addCons(constrain)

opt.quicksum?

Docstring:
add linear expressions and constants much faster than Python's sum
by avoiding intermediate data structures and adding terms inplace

In [21]:
# 约束2, 每个网格内有且只有一个数字
for r in range(N):
    for c in range(N):
        constrain = opt.quicksum(variables[r, c, k] for k in constants) == 1
        model.addCons(constrain)

In [22]:
# 约束3, 行列约束
for i in range(N):
    for k in constants:
        # 行约束
        r_constrain = opt.quicksum(variables[i, c, k] for c in range(N)) == 1
        model.addCons(r_constrain)
        # 列约束
        c_constrain = opt.quicksum(variables[r, i, k] for r in range(N)) == 1
        model.addCons(c_constrain) 

In [23]:
# 约束4, 子宫格约束              
for row in range(n):
    for col in range(n):
        for k in constants:
            constrain = opt.quicksum(variables[r + n * row, c + n * col, k] for r in range(n) for c in range(n)) == 1
            model.addCons(constrain)

以上四大约束则对应着上述的数学模型的转换

In [24]:
# 无目标函数
model.optimize()

model.getVal?

Docstring:
Retrieve the value of the given variable or expression in the best known solution.
Can only be called after solving is completed.

:param Expr expr: polynomial expression to query the value of

Note: a variable is also an expression

In [25]:
if model.getStatus() == 'optimal':
    print('sudoku solution:')
    # 预先构造一个数独容器用于存放解析结果
    solution = [[0 for _ in range(N)] for _ in range(N)]
    for (r, c, k) in variables.keys():
        if model.getVal(variables[r, c, k]) == 1:
            solution[r][c] = k
    # 绘制数独结果
    sudo_data = (''.join(f'{"| " if c % 3 == 0 else " "}{solution[r][c]}{" " if matrix[r][c] == 0 else "*"}' for c in range(N)) + '|' + ('\n'+ '-' * 31 if (r + 1) % 3 == 0 and r != N - 1 else '') for r in range(N))
    print('\n'.join(sudo_data))
else: print('warning: no solution')

sudoku solution:
| 8* 1  2 | 7  5  3 | 6  4  9 |
| 9  4  3*| 6* 8  2 | 1  7  5 |
| 6  7* 5 | 4  9* 1 | 2* 8  3 |
-------------------------------
| 1  5* 4 | 2  3  7*| 8  9  6 |
| 3  6  9 | 8  4* 5*| 7* 2  1 |
| 2  8  7 | 1* 6  9 | 5  3* 4 |
-------------------------------
| 5  2  1*| 9  7  4 | 3  6* 8*|
| 4  3  8*| 5* 2  6 | 9  1* 7 |
| 7  9* 6 | 3  1  8 | 4* 5  2 |
